# 1

<div style="color:white;display:fill;border-radius:8px;
            background-color:#FFE5B4;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2B0245'> 1 | Importing the libraries</span></b></b></p>
</div>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U spacy
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 84.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
import spacy
from spacy.tokens import DocBin

In [ ]:
from tqdm import tqdm
import json
import pickle
import random
import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

# 2

<div style="color:white;display:fill;border-radius:8px;
            background-color:#FFE5B4;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2B0245'> 2 | Losding the data</span></b></b></p>
</div>

In [ ]:
#train_data = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Resume Analyser/data/data_summarise_info/training/train_data.pkl','rb'))
train_data = json.load(open('/content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/data/data_summarise_info/training/train_data.json','r'))

In [ ]:
# Let's see the data
train_data[0]

['Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

# 3

<div style="color:white;display:fill;border-radius:8px;
            background-color:#FFE5B4;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2B0245'> 3 | Training the data on the model</span></b></b></p>
</div>

We will first load a black SpaCy english model. Then we will write a function which will take the training data as the input. In the function, first we will add a ner i.e. Named Entity Recognition in the last position in the pipeline. Then we will add our custom labels in the pipeline.

In [ ]:
pwd

'/content'

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_config/base_config.cfg /content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_config/config.cfg

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
def get_spacy_doc(file, data):

  nlp = spacy.blank("en")
  db = DocBin()

  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label = label, alignment_mode = 'strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + '\n'
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [ ]:
from sklearn.model_selection import train_test_split
train, dev = train_test_split(train_data, test_size = 0.2, random_state = 42)

In [ ]:
file = open('/content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_model/train_file.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('/content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_model/train.spacy')

db = get_spacy_doc(file, dev)
db.to_disk('/content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_model/dev.spacy')

file.close()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
100%|██████████| 40/40 [00:00<00:00, 95.62it/s] 


In [ ]:
!python -m spacy train /content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_config/config.cfg --output /content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_model/output --paths.train /content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_model/train.spacy --paths.dev /content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_model/dev.spacy --gpu-id 0

ℹ Saving to output directory:
/content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_model/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/

# 4

<div style="color:white;display:fill;border-radius:8px;
            background-color:#FFE5B4;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#2B0245'> 4 | Test Model testing</span></b></b></p>
</div>


Let's check out how our model is performing.
For this we will pass a new resume to this model.

In [ ]:
'''# Installing PyMuPDF for getting the text data from the resume pdf'''
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 49.0 MB/s eta 0:00:00


In [ ]:
import re
import sys, fitz

# Compile the regex pattern to match "Page" followed by a number, "of," and another number
pattern = r"Page \d+ of \d+"

fname = '/content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/data/data_summarise_info/test/Alice Clark CV.pdf'

doc = fitz.open(fname)
pages_text = []
# Iterate through each page
for page_num in range(len(doc)):
    # Get the page
    page = doc.load_page(page_num)
    # Extract text from the page
    text = page.get_text()
    # Append the text to the list
    pages_text.append(text)
# Close the document
doc.close()

text = ""
for page in pages_text:
    text = text + str(page)
    page = re.sub(pattern, "", page)
    print(page)

tx = " ".join(text.split('\n'))  # for removing the next line character '/n'
tx1 = tx.replace("/", "\n")
tx2 = tx1.replace("•", "\n\t")
print(tx2.replace(".", "\n"))

Alice Clark 
AI / Machine Learning 
 
Delhi, India Email me on Indeed 
• 
20+ years of experience in data handling, design, and development 
• 
Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to 
data warehousing and business intelligence 
• 
Database: Experience in database designing, scalability, back-up and recovery, writing and 
optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. 
Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, 
Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake 
analytics(U-SQL) 
Willing to relocate anywhere 
 
WORK EXPERIENCE 
Software Engineer 
Microsoft – Bangalore, Karnataka 
January 2000 to Present 
1. Microsoft Rewards Live dashboards: 
Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping 
online. Microsoft Rewards members can earn points when searching with Bing, bro

In [ ]:
tx2

'Alice Clark  AI \n Machine Learning    Delhi, India Email me on Indeed  \n\t  20+ years of experience in data handling, design, and development  \n\t  Data Warehouse: Data analysis, star\nsnow flake scema data modelling and design specific to  data warehousing and business intelligence  \n\t  Database: Experience in database designing, scalability, back-up and recovery, writing and  optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure,  Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake  analytics(U-SQL)  Willing to relocate anywhere    WORK EXPERIENCE  Software Engineer  Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching wi

In [ ]:
for page in pages_text:
  model = spacy.load('/content/drive/MyDrive/Colab_Notebooks/Resume_Analyser/2.version/parser_ner_model/output/model-last')
  doc = model(page)
  store = []
  for ent in doc.ents:
    if str(ent) in store:
      continue
    print(f'{ent.label_.upper():{30}} >>>>> {ent.text}')
    store.append(str(ent))
  print("-----")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We

In [ ]:
"""
NAME                          - Alice Clark
LOCATION                      - Delhi
DESIGNATION                   - Stream Analytics
DESIGNATION                   - Software Engineer
COMPANIES WORKED AT           - Microsoft –
DEGREE                        - Indian Institute of Technology – Mumbai
SKILLS                        - Machine Learning, Natural Language Processing, and Big Data Handling    ADDITIONAL INFORMATION  Professional Skills  • Excellent analytical, problem solving, communication, knowledge transfer and interpersonal"""